In [ ]:
import getpass
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "RAG Demo 1"
os.environ["LANGSMITH_ENDPOINT"] = "https://eu.api.smith.langchain.com"
os.environ["USER_AGENT"] = "MyRagDemo/1.0"

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API key: ")

# RAG DEMO 1

Allereerst maken we gebruik van een chat model, in dit geval die van OpenAi.

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Vervolgens moeten we een provider kiezen voor onze embeddings model, ook hier OpenAI.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

En een store voor onze vectors. Voor de demo is dit een In Memory store, maar kan ook Elasticsearch zijn o.i.d.

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

### Inladen van de documenten
Inladen van een tekst. In dit geval de Silmarillion van Tolkien.
Toch best een lastig boek waar zelfs de meest die-hard fans nog massa's vragen over hebben.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep the text, skip the rest of the HTML.
# Other example: parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
bs4_strainer = bs4.SoupStrainer(['pre'])
loader = WebBaseLoader(
    web_paths=("https://archive.org/stream/TheSilmarillionIllustratedJ.R.R.TolkienTedNasmith/The%20Silmarillion%20%28Illustrated%29%20-%20J.%20R.%20R.%20Tolkien%3B%20Ted%20Nasmith%3B_djvu.txt",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")
#print(docs[0].page_content[:500])

### Splitsen van de documenten
Ik gebruik hier een algemene splitter. Je hebt specifieke splitters voor code, academische papers, etc.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split text into {len(all_splits)} sub-documents.")

### Opslaan van de chuncks
Omdat we het embeddingsmodel gekoppeld hebben worden de embeddings gegenereerd en als vector opgeslagen.

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

### Aan de slag

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

### RAG applicatie

We gebruiken een State class om wat zaken bij te houden. In dit geval vraag, context en antwoord.
Twee functies helpen bij de stappen voor het ophalen en het genereren van het antwoord.

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Grappig: de graph ondersteunt Mermaid Diagrams, dus we kunnen onze simpele flow visualiseren:

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

### Uitvoeren

In [ ]:
response = graph.invoke({"question": "What is Tolkien's core message in the Silmarillion?"})
print(response["answer"])